In [0]:
# Databricks notebook source
dbutils.fs.mount(
  source = "wasbs://bronze@sanilayanalytics.blob.core.windows.net",
  mount_point = "/mnt/superstore",
  extra_configs = {"fs.azure.account.key.sanilayanalytics.blob.core.windows.net":"STORAGEKEY"})


Out[7]: True

In [0]:

dbutils.fs.ls("/mnt/superstore/")

Out[9]: [FileInfo(path='dbfs:/mnt/superstore/global_superstore_data.parquet', name='global_superstore_data.parquet', size=5264489, modificationTime=1754579932000)]

In [0]:
# Read raw data from bronze layer
df = spark.read.parquet("/mnt/superstore/global_superstore_data.parquet")
df.printSchema()




root
 |-- row_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- ship_date: string (nullable = true)
 |-- ship_mode: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- segment: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- market: string (nullable = true)
 |-- region: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sub-category: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- sales: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- discount: string (nullable = true)
 |-- profit: string (nullable = true)
 |-- shipping_cost: string (nullable = true)
 |-- order_priority: string (nullable = true)



In [0]:
# Import necessary functions for string manipulation, date conversion, and type casting
from pyspark.sql.functions import col, trim, regexp_replace, substring_index, to_date
from pyspark.sql.types import DecimalType, IntegerType

# Data cleaning and type normalization
df_cleaned = (
    df
    # 1️⃣ Remove extra spaces from 'order_date' and 'ship_date'
    .withColumn("order_date", trim(regexp_replace(col("order_date"), " +", " ")))
    .withColumn("ship_date", trim(regexp_replace(col("ship_date"), " +", " ")))

    # 2️⃣ Keep only the first part of the date string (before any space)
    .withColumn("order_date", substring_index(col("order_date"), " ", 1))
    .withColumn("ship_date", substring_index(col("ship_date"), " ", 1))

    # 3️⃣ Convert string columns to actual DateType with the format 'dd-MM-yyyy'
    .withColumn("order_date", to_date(col("order_date"), "dd-MM-yyyy"))
    .withColumn("ship_date", to_date(col("ship_date"), "dd-MM-yyyy"))

    # 4️⃣ Cast numeric columns to appropriate numeric types
    .withColumn("sales", col("sales").cast(DecimalType(10, 2)))       # Up to 10 digits, 2 decimal places
    .withColumn("quantity", col("quantity").cast(IntegerType()))      # Whole numbers only
    .withColumn("discount", col("discount").cast(DecimalType(4, 2)))  # Max 99.99
    .withColumn("profit", col("profit").cast(DecimalType(10, 2)))     # Up to 10 digits, 2 decimal places
    .withColumn("shipping_cost", col("shipping_cost").cast(DecimalType(10, 2)))  # Shipping cost as decimal
)

# Print schema to confirm data types after cleaning
df_cleaned.printSchema()



root
 |-- row_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_date: date (nullable = true)
 |-- ship_date: date (nullable = true)
 |-- ship_mode: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- segment: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- market: string (nullable = true)
 |-- region: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sub-category: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- sales: decimal(10,2) (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- discount: decimal(4,2) (nullable = true)
 |-- profit: decimal(10,2) (nullable = true)
 |-- shipping_cost: decimal(10,2) (nullable = true)
 |-- order_priority: string (nullable = t

In [0]:
# I want to see the data
df_superstore = df_casted

display(df_cleaned.limit(10))

row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,city,state,country,postal_code,market,region,product_id,category,sub-category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority
32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,United States,10024,US,East,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wireless Headset System,2309.65,7,0.00,762.18,933.57,Critical
26341,IN-2013-77878,2013-02-05,2013-02-07,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,Australia,null,APAC,Oceania,FUR-CH-10003950,Furniture,Chairs,Novimex Executive Leather Armchair- Black,3709.40,9,0.10,-288.77,923.63,Critical
25330,IN-2013-71249,2013-10-17,2013-10-18,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,Australia,null,APAC,Oceania,TEC-PH-10004664,Technology,Phones,Nokia Smart Phone- with Caller ID,5175.17,9,0.10,919.97,915.49,Medium
13524,ES-2013-1579342,2013-01-28,2013-01-30,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,Germany,null,EU,Central,TEC-PH-10004583,Technology,Phones,Motorola Smart Phone- Cordless,2892.51,5,0.10,-96.54,910.16,Medium
47221,SG-2013-4320,2013-11-05,2013-11-06,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,Senegal,null,Africa,Africa,TEC-SHA-10000501,Technology,Copiers,Sharp Wireless Fax- High-Speed,2832.96,8,0.00,311.52,903.04,Critical
22732,IN-2013-42360,2013-06-28,2013-07-01,Second Class,JM-15655,Jim Mitchum,Corporate,Sydney,New South Wales,Australia,null,APAC,Oceania,TEC-PH-10000030,Technology,Phones,Samsung Smart Phone- with Caller ID,2862.68,5,0.10,763.28,897.35,Critical
30570,IN-2011-81826,2011-11-07,2011-11-09,First Class,TS-21340,Toby Swindell,Consumer,Porirua,Wellington,New Zealand,null,APAC,Oceania,FUR-CH-10004050,Furniture,Chairs,Novimex Executive Leather Armchair- Adjustable,1822.08,4,0.00,564.84,894.77,Critical
31192,IN-2012-86369,2012-04-14,2012-04-18,Standard Class,MB-18085,Mick Brown,Consumer,Hamilton,Waikato,New Zealand,null,APAC,Oceania,FUR-TA-10002958,Furniture,Tables,Chromcraft Conference Table- Fully Assembled,5244.84,6,0.00,996.48,878.38,High
40155,CA-2014-135909,2014-10-14,2014-10-21,Standard Class,JW-15220,Jane Waco,Corporate,Sacramento,California,United States,95823,US,West,OFF-BI-10003527,Office Supplies,Binders,Fellowes PB500 Electric Punch Plastic Comb Binding Machine with Manual Bind,5083.96,5,0.20,1906.49,867.69,Low
40936,CA-2012-116638,2012-01-28,2012-01-31,Second Class,JH-15985,Joseph Holt,Consumer,Concord,North Carolina,United States,28027,US,South,FUR-TA-10000198,Furniture,Tables,Chromcraft Bull-Nose Wood Oval Conference Tables & Bases,4297.64,13,0.40,-1862.31,865.74,Critical


In [0]:
from pyspark.sql.functions import col, countDistinct, dense_rank
from pyspark.sql.window import Window

# -----------------------------------
# 1️⃣ Check if product_id has multiple product_names
# -----------------------------------
# Goal: Identify if the same product_id is linked to more than one product_name
product_check = (
    df_cleaned
    .groupBy("product_id")
    .agg(countDistinct("product_name").alias("distinct_product_names"))
    .orderBy(col("distinct_product_names").desc())
)

display(product_check.limit(10))  # Show top cases with highest distinct names


product_id,distinct_product_names
OFF-PA-10004673,4
FUR-FU-10004960,3
OFF-AR-10003829,3
OFF-BI-10004654,3
OFF-AR-10003651,3
FUR-BO-10002204,3
TEC-AC-10004334,3
OFF-BI-10004632,3
TEC-AC-10002842,3
OFF-SU-10003629,3


In [0]:
# -----------------------------------
# 2️⃣ Generate surrogate keys for each unique product_name
# -----------------------------------
# Get distinct product names
distinct_products = df_cleaned.select("product_name").distinct()

# Assign surrogate keys in a consistent order
window_spec = Window.orderBy("product_name")
product_keys = (
    distinct_products
    .withColumn("product_key", dense_rank().over(window_spec))
)

# ✅ Show the mapping to prove surrogate keys were generated
display(product_keys.limit(10))


product_name,product_key
"""While you Were Out"""" Message Book- One Form per Page""",1
#10 Gummed Flap White Envelopes- 100/Box,2
#10 Self-Seal White Envelopes,3
#10 White Business Envelopes-4 1/8 x 9 1/2,4
"#10- 4 1/8"" x 9 1/2"""" Recycled Envelopes""",5
"#10- 4 1/8"" x 9 1/2"""" Security-Tint Envelopes""",6
"#10-4 1/8"" x 9 1/2"""" Premium Diagonal Seam Envelopes""",7
#6 3/4 Gummed Flap White Envelopes,8
"1.7 Cubic Foot Compact ""Cube"""" Office Refrigerators""",9
"1/4 Fold Party Design Invitations & White Envelopes- 24 8-1/2"" X 11"""" Cards- 25 Env./Pack""",10


In [0]:
# 3️⃣ Create flat file by replacing old product_id with surrogate key
# -----------------------------------
df_flat = (
    df_cleaned.drop("product_id")  # Remove old product_id
    .join(product_keys, on="product_name", how="left")  # Add surrogate key
)

# Optional: Reorder columns so product_key appears first
cols = ["product_key"] + [c for c in df_flat.columns if c != "product_key"]
df_flat = df_flat.select(cols)

display(df_flat.limit(10))


product_key,product_name,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,city,state,country,postal_code,market,region,category,sub-category,sales,quantity,discount,profit,shipping_cost,order_priority
2751,Plantronics CS510 - Over-the-Head monaural Wireless Headset System,32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,United States,10024,US,East,Technology,Accessories,2309.65,7,0.00,762.18,933.57,Critical
2526,Novimex Executive Leather Armchair- Black,26341,IN-2013-77878,2013-02-05,2013-02-07,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,Australia,null,APAC,Oceania,Furniture,Chairs,3709.40,9,0.10,-288.77,923.63,Critical
2503,Nokia Smart Phone- with Caller ID,25330,IN-2013-71249,2013-10-17,2013-10-18,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,Australia,null,APAC,Oceania,Technology,Phones,5175.17,9,0.10,919.97,915.49,Medium
2415,Motorola Smart Phone- Cordless,13524,ES-2013-1579342,2013-01-28,2013-01-30,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,Germany,null,EU,Central,Technology,Phones,2892.51,5,0.10,-96.54,910.16,Medium
3159,Sharp Wireless Fax- High-Speed,47221,SG-2013-4320,2013-11-05,2013-11-06,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,Senegal,null,Africa,Africa,Technology,Copiers,2832.96,8,0.00,311.52,903.04,Critical
2992,Samsung Smart Phone- with Caller ID,22732,IN-2013-42360,2013-06-28,2013-07-01,Second Class,JM-15655,Jim Mitchum,Corporate,Sydney,New South Wales,Australia,null,APAC,Oceania,Technology,Phones,2862.68,5,0.10,763.28,897.35,Critical
2525,Novimex Executive Leather Armchair- Adjustable,30570,IN-2011-81826,2011-11-07,2011-11-09,First Class,TS-21340,Toby Swindell,Consumer,Porirua,Wellington,New Zealand,null,APAC,Oceania,Furniture,Chairs,1822.08,4,0.00,564.84,894.77,Critical
922,Chromcraft Conference Table- Fully Assembled,31192,IN-2012-86369,2012-04-14,2012-04-18,Standard Class,MB-18085,Mick Brown,Consumer,Hamilton,Waikato,New Zealand,null,APAC,Oceania,Furniture,Tables,5244.84,6,0.00,996.48,878.38,High
1430,Fellowes PB500 Electric Punch Plastic Comb Binding Machine with Manual Bind,40155,CA-2014-135909,2014-10-14,2014-10-21,Standard Class,JW-15220,Jane Waco,Corporate,Sacramento,California,United States,95823,US,West,Office Supplies,Binders,5083.96,5,0.20,1906.49,867.69,Low
911,Chromcraft Bull-Nose Wood Oval Conference Tables & Bases,40936,CA-2012-116638,2012-01-28,2012-01-31,Second Class,JH-15985,Joseph Holt,Consumer,Concord,North Carolina,United States,28027,US,South,Furniture,Tables,4297.64,13,0.40,-1862.31,865.74,Critical


In [0]:
# 4️⃣ Verify surrogate keys have unique product_name mapping
# -----------------------------------
check_df = (
    df_flat
    .groupBy("product_key")
    .agg(countDistinct("product_name").alias("distinct_names_per_key"))
    .filter(col("distinct_names_per_key") > 1)  # Keep only problem cases
)

# If this returns no rows → clean mapping
check_df.show(truncate=False)


+-----------+----------------------+
|product_key|distinct_names_per_key|
+-----------+----------------------+
+-----------+----------------------+



In [0]:
# 5️⃣ Save the flat file to Silver layer as Parquet
# -----------------------------------
# Target path in the Silver container
silver_path = "abfss://silver@sanilayanalytics.dfs.core.windows.net/global_superstore_silver"

# Write df_flat to the Silver layer in Parquet format
# 'overwrite' mode ensures fresh data replaces any old file
df_flat.write.mode("overwrite").parquet(silver_path)

print("✅ Flat file successfully saved to Silver layer.")

# 6️⃣ Read back the saved Parquet file from Silver layer for verification
# -----------------------------------
df_silver = spark.read.parquet(silver_path)

# Display a sample of the data from Silver layer
display(df_silver.limit(10))


✅ Flat file successfully saved to Silver layer.


product_name,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,city,state,country,postal_code,market,region,category,sub-category,sales,quantity,discount,profit,shipping_cost,order_priority,product_key
Plantronics CS510 - Over-the-Head monaural Wireless Headset System,32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,United States,10024,US,East,Technology,Accessories,2309.65,7,0.00,762.18,933.57,Critical,2751
Novimex Executive Leather Armchair- Black,26341,IN-2013-77878,2013-02-05,2013-02-07,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,Australia,null,APAC,Oceania,Furniture,Chairs,3709.40,9,0.10,-288.77,923.63,Critical,2526
Nokia Smart Phone- with Caller ID,25330,IN-2013-71249,2013-10-17,2013-10-18,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,Australia,null,APAC,Oceania,Technology,Phones,5175.17,9,0.10,919.97,915.49,Medium,2503
Motorola Smart Phone- Cordless,13524,ES-2013-1579342,2013-01-28,2013-01-30,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,Germany,null,EU,Central,Technology,Phones,2892.51,5,0.10,-96.54,910.16,Medium,2415
Sharp Wireless Fax- High-Speed,47221,SG-2013-4320,2013-11-05,2013-11-06,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,Senegal,null,Africa,Africa,Technology,Copiers,2832.96,8,0.00,311.52,903.04,Critical,3159
Samsung Smart Phone- with Caller ID,22732,IN-2013-42360,2013-06-28,2013-07-01,Second Class,JM-15655,Jim Mitchum,Corporate,Sydney,New South Wales,Australia,null,APAC,Oceania,Technology,Phones,2862.68,5,0.10,763.28,897.35,Critical,2992
Novimex Executive Leather Armchair- Adjustable,30570,IN-2011-81826,2011-11-07,2011-11-09,First Class,TS-21340,Toby Swindell,Consumer,Porirua,Wellington,New Zealand,null,APAC,Oceania,Furniture,Chairs,1822.08,4,0.00,564.84,894.77,Critical,2525
Chromcraft Conference Table- Fully Assembled,31192,IN-2012-86369,2012-04-14,2012-04-18,Standard Class,MB-18085,Mick Brown,Consumer,Hamilton,Waikato,New Zealand,null,APAC,Oceania,Furniture,Tables,5244.84,6,0.00,996.48,878.38,High,922
Fellowes PB500 Electric Punch Plastic Comb Binding Machine with Manual Bind,40155,CA-2014-135909,2014-10-14,2014-10-21,Standard Class,JW-15220,Jane Waco,Corporate,Sacramento,California,United States,95823,US,West,Office Supplies,Binders,5083.96,5,0.20,1906.49,867.69,Low,1430
Chromcraft Bull-Nose Wood Oval Conference Tables & Bases,40936,CA-2012-116638,2012-01-28,2012-01-31,Second Class,JH-15985,Joseph Holt,Consumer,Concord,North Carolina,United States,28027,US,South,Furniture,Tables,4297.64,13,0.40,-1862.31,865.74,Critical,911
